In [4]:
from os import walk
import os
from texture_synthesis.feature_extractor import FeatureExtractor
from texture_synthesis.function import *
import torch
from PIL import Image
import numpy as np
from texture_synthesis.style_features_manipulation import *

In [5]:
device=torch.device("cuda" if (torch.cuda.is_available()) else 'cpu')
FE = FeatureExtractor().to(device)

In [8]:
def check_separation_accuracy(path1, path2, ds_size=None):
    filenames1 = next(walk(path1), (None, None, []))[2]
    filenames2 = next(walk(path2), (None, None, []))[2]

    images1 = [Image.open(path1 + '/' + filename).convert("RGB").resize((256, 256)) for filename in filenames1]
    images2 = [Image.open(path2 + '/' + filename).convert("RGB").resize((256, 256)) for filename in filenames2]

    style_tensor_set1 = [FE.get_style_representation(image_to_tensor(img.resize((256, 256)), False).to(device), K=2) for img in images1]
    style_tensor_set2 = [FE.get_style_representation(image_to_tensor(img.resize((256, 256)), False).to(device), K=2) for img in images2]

    if ds_size is not None:
        d, svm = style_attribute_extraction_svm(style_tensor_set1[:ds_size], style_tensor_set2[:ds_size], C=1)
    else:
        d, svm = style_attribute_extraction_svm(style_tensor_set1, style_tensor_set2, C=1)

    class1 = np.logical_not(svm.predict(torch.stack(style_tensor_set1, 0).cpu().numpy())).astype(int)
    class2 = svm.predict(torch.stack(style_tensor_set2, 0).cpu().numpy())

    predictions = np.append(class1, class2)
    print(len(predictions[predictions == 1]) / len(predictions) * 100)

## Curls

In [9]:
path1 = 'C:/Users/М-Видео/Desktop/grad work/project/images/marble'
path2 = 'C:/Users/М-Видео/Desktop/grad work/project/images/curly marble'

print('All:')
check_separation_accuracy(path1, path2)
print('10:')
check_separation_accuracy(path1, path2, 10)
print('1:')
check_separation_accuracy(path1, path2, 1)

All:
100.0
10:
81.94444444444444
1:
52.77777777777778


## Flowers

In [10]:
path1 = 'C:/Users/М-Видео/Desktop/grad work/project/images/grass'
path2 = 'C:/Users/М-Видео/Desktop/grad work/project/images/grass with flowers'

print('All:')
check_separation_accuracy(path1, path2)
print('10:')
check_separation_accuracy(path1, path2, 10)
print('1:')
check_separation_accuracy(path1, path2, 1)

All:
100.0
10:
90.66666666666666
1:
66.66666666666666


## Grass

In [11]:
path1 = 'C:/Users/М-Видео/Desktop/grad work/project/images/rocks'
path2 = 'C:/Users/М-Видео/Desktop/grad work/project/images/rocks_with_grass'

print('All:')
check_separation_accuracy(path1, path2)
print('10:')
check_separation_accuracy(path1, path2, 10)
print('1:')
check_separation_accuracy(path1, path2, 1)

All:
100.0
10:
93.20388349514563
1:
80.58252427184466
